# Parse PDFs with Docling

In [4]:
from utils import get_companies_dict

companies_dict = get_companies_dict(r"../data/round2/subset.json")

In [2]:
# only keep subset for dev
companies = ["Ziff Davis, Inc.", ""]
companies_dict = {company: data for company, data in companies_dict.items() if company in companies}

In [ ]:
# Potential for parallelization for files?
import os
from docling.document_converter import DocumentConverter
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm


converter = DocumentConverter()  # 20 sec for setting up

preproc_logs = {"failed": [], "skipped": []}


def process_company(company, data):
    pdf_path = f"../data/round2/pdfs/{data['sha1']}.pdf"
    docling_md_path = f"../data/docling_md/round2/docling_{data['sha1']}.md"

    if not os.path.exists(docling_md_path):
        print(f"Processing {company}")
        try:
            result = converter.convert(pdf_path)
            with open(docling_md_path, "w", encoding="utf-8") as file:
                file.write(result.document.export_to_markdown())
            # Return None error to indicate success.
            return (company, None)
        except Exception as e:
            print(f"Error processing {company}: {e}")
            return (company, e)
    else:
        print(f"Skipping {company}")
        return (company, "skipped")

# Use ThreadPoolExecutor to process companies concurrently.
with ThreadPoolExecutor(max_workers=1) as executor:
    # Submit all tasks
    futures = {executor.submit(process_company, company, data): company
               for company, data in companies_dict.items()}

    # Process tasks as they complete.
    for future in tqdm(as_completed(futures), total=len(futures)):
        company, result = future.result()
        if result is None:
            continue  # Successful processing.
        elif result == "skipped":
            preproc_logs["skipped"].append(company)
        else:
            preproc_logs["failed"].append({"company_id": company, "error": result})

if preproc_logs["failed"]:
    print(f"Nr of failed companies: {len(preproc_logs['failed'])}")


# RUNTIMES:
# REIT: execution for 7.4MB with 97 pages -> 5 min
# Yellow Pages: execution for 1.4MB with 77 pages -> 4 min
# Calyxt: execution for 0.7MB with 88 pages -> 3min
# TODO check on google colab

# NOTES:
# brackets in tables indicate negative values
# parsing issues (see current system prompt)

Skipping ACRES Commercial Realty Corp.
Skipping Aptevo Therapeutics Inc.


  0%|          | 0/100 [00:00<?, ?it/s]

Skipping Downer EDI Limited
Skipping Odyssey Gold Limited
Skipping NextNav Inc.
Skipping Peako Limited
Skipping Mosaic Brands Limited
Skipping Aurora Innovation, Inc.
Skipping Crombie REIT
Skipping Medallion Financial Corp.
Skipping Enact Holdings, Inc.
Skipping BetMakers Technology Group Ltd
Skipping OFX Group Limited
Skipping FNCB Bancorp, Inc.
Skipping Celldex Therapeutics, Inc.
Skipping SIG plc
Skipping Motability Operations Group plc
Skipping BCB Bancorp, Inc.
Skipping 1-800-FLOWERS.COM, INC.
Skipping Weis Markets, Inc.
Skipping Odyssey Group Holdings, Inc.
Skipping Blue Apron Holdings, Inc.
Skipping Ocugen, Inc.
Skipping Rectifier Technologies Ltd
Skipping Structural Monitoring Systems Plc
Skipping Origin Bancorp, Inc.
Skipping Liberty Broadband Corporation
Skipping Incyte Corporation
Skipping Guaranty Bancshares, Inc.
Skipping Infinera Corporation
Skipping MainStreet Bancshares, Inc.
Skipping Starvest plc
Skipping INMUNE BIO INC.
Skipping Bionano Genomics, Inc.
Skipping AstraZen

 41%|████      | 41/100 [1:18:11<1:52:31, 114.44s/it]

Skipping Sonic Automotive, Inc.
Processing Commerzbank


...

Processing archTIS Limited


 71%|███████   | 71/100 [4:53:35<3:34:50, 444.50s/it]

Processing Beazley plc


 72%|███████▏  | 72/100 [5:22:22<6:26:55, 829.11s/it]

Processing Poste Italiane
